In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from os import listdir
from sklearn.model_selection import train_test_split
import torch
from skopt import gp_minimize

In [3]:
from sktime.performance_metrics.forecasting import smape_loss
from lightgbm import LGBMRegressor
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.compose import make_reduction
from statsmodels.tsa.filters.hp_filter import hpfilter

In [4]:
PATH = r'C:\Users\heylu\Documents\github\ansiedade-e-pandemia\data'

In [5]:
df = pd.DataFrame()
for year in listdir(PATH):
    for f in listdir(PATH + '\\' + year):
        dataset_from_files_in_path = pd.read_csv(PATH + '\\' + year + '\\' + f)
        df = pd.concat([df, dataset_from_files_in_path])
    df.drop(columns='Unnamed: 0', inplace=True)

In [6]:
df.drop(columns='Unnamed: 0.1', inplace=True)
df.drop(columns='Unnamed: 0.1.1', inplace=True)

In [7]:
frequency = df[['date', 'labels']]
frequency['quantidade'] = 1

<ipython-input-7-18ce582a2750>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frequency['quantidade'] = 1


In [8]:
frequency.to_csv(r'C:\Users\heylu\Documents\github\A Study Case About Mental Health Tweets in Pandemic Times\notebooks\EDA\frequency.csv')

In [9]:
frequency_per_day = frequency.groupby('date').sum()
frequency_per_day.index = pd.to_datetime(frequency_per_day.index)
frequency_per_day.index.freq = 'D'
frequency_per_month = frequency_per_day.groupby(pd.Grouper(freq='M')).sum()
frequency_per_month.drop(columns='labels', inplace=True)
frequency_per_day.drop(columns='labels', inplace=True)

In [10]:
fig = px.line(frequency_per_day)
fig

In [11]:
fig = px.line(frequency_per_month)
fig

In [12]:
fig = px.bar(df['labels'].value_counts() / len(df['labels']) * 100)

fig

In [13]:
frequency_per_label = frequency.groupby(['labels', 'date']).sum()
frequency_per_label = frequency_per_label.reset_index()

In [14]:
fig = px.line(frequency_per_label, x='date', y='quantidade', facet_row='labels', height=800, width=800)
fig.update_yaxes(matches=None)
fig

In [15]:
frequency_per_label = frequency_per_label.set_index('date')
frequency_per_label.index = pd.to_datetime(frequency_per_label.index)
frequency_per_label_per_month = frequency_per_label.groupby(['labels',pd.Grouper(freq='M')]).sum()
frequency_per_label_per_month = frequency_per_label_per_month.reset_index()

In [16]:
fig = px.line(frequency_per_label_per_month, x='date', y='quantidade', facet_row='labels', height=800, width=800)
fig.update_yaxes(matches=None)
fig

In [17]:
fig = px.bar(frequency_per_day.groupby(pd.Grouper(freq='Y')).sum())

fig

In [18]:
fig = px.bar(frequency_per_day[frequency_per_day.index < '2021-01-01'].groupby(pd.Grouper(freq='Y')).sum().pct_change())

fig

In [19]:
fig = px.bar(frequency_per_month.pct_change())

fig

In [21]:
frequency_per_day

,quantidade
date,
2018-01-01,1353
2018-01-02,2207
2018-01-03,2473
2018-01-04,2302
2018-01-05,2149
...,...
2021-04-26,16949
2021-04-27,18073
2021-04-28,15501


In [23]:
fh = np.arange(1, 180 + 1)
y = frequency_per_day
y = y['quantidade']
y.index.freq = 'D'
SEED = 42

In [27]:
#cv = ExpandingWindowSplitter(initial_window=200)
#cross_val = evaluate(forecaster=reg, y=y, cv=cv, strategy="refit", return_data=True)
#cross_val['test_sMAPE'].to_csv(r'C:\Users\heylu\Documents\github\Aansiedade-e-pandemia\notebooks\EDA\cross_val_sMAPE.csv')

In [28]:
#reg.fit(y)
#y_pred = reg.predict(fh).round()
#y_pred = y_pred[y_pred.index < '2021-10-01']
#y_pred.to_csv(r'C:\Users\heylu\Documents\github\ansiedade-e-pandemia\notebooks\EDA\predictions.csv')

In [29]:
#def search_hyperparams(params):
#    SEED = 42
#
#    learning_rate = params[0]
#    num_leaves = params[1]
#    min_child_samples = params[2]
#    subsample = params[3]
#    colsample_bytree = params[4]
#    n_estimators = params[5]
#
#    print(params, '\n')
#    
#    model = LGBMRegressor(random_state=SEED,
#    learning_rate = learning_rate, 
#    num_leaves = num_leaves, 
#    min_child_samples = min_child_samples, 
#    subsample = subsample,
#    colsample_bytree = colsample_bytree,
#    subsample_freq=1,
#    n_estimators=n_estimators)
#
#    reg = make_reduction(estimator=model, window_length=180)
#    cv = ExpandingWindowSplitter(initial_window=200)
#    cross_val = evaluate(forecaster=reg, y=y, cv=cv, strategy="refit", return_data=True)
#    cross_val = cross_val['test_sMAPE'].mean()
#
#    return cross_val
#
#space = [(1e-3, 1e-1, 'log-uniform'), #learning rate
#(2, 128), #num_leaves
#(1, 100), #min_child_samples
#(0.05, 1.0), #subsample
#(0.1, 1.0), #colsample_bytree
#(100, 1000)] #n_estimators
#
#result = gp_minimize(search_hyperparams, 
#space,
#random_state=SEED, 
#verbose=1,
#n_calls = 20,
#n_random_starts=10)
#learning_rate, num_leaves, min_child_samples, subsample, colsample_bytree, n_estimators = result.x 

In [25]:
model = LGBMRegressor(random_state=SEED,
    learning_rate = 0.02416872015582117, 
    num_leaves = 59, 
    min_child_samples = 1, 
    subsample = 0.8332979201831602,
    colsample_bytree = 0.7546809603712795,
    subsample_freq=1,
    n_estimators=1000)
reg = make_reduction(estimator=model, window_length=180)

In [32]:
smape = pd.read_csv(r'C:\Users\heylu\Documents\github\A Study Case About Mental Health Tweets in Pandemic Times\notebooks\EDA\cross_val_sMAPE.csv')
smape['test_sMAPE'].mean()

In [28]:
reg.fit(y)
y_pred = reg.predict(fh).round()
y_pred = y_pred[y_pred.index < '2021-10-01']
#y_pred.to_csv(r'C:\Users\heylu\Documents\github\ansiedade-e-pandemia\notebooks\EDA\predictions.csv')
_, trend = hpfilter(y_pred)
trend.to_csv(r'C:\Users\heylu\Documents\github\A Study Case About Mental Health Tweets in Pandemic Times\streamlit\trend.csv')

In [29]:
fig = px.line(trend)
fig

In [38]:
media_de_casos_por_mes = 539152.8